In which I attempt to use the Camera controller to play nicely with the camera despite being agnostic about the units.



In [1]:
from SpheroController.tracablesphero import TraceableSphero

In [2]:
import sphero

# Controller Functions
import time

import os
import json
from pprint import pprint
import math

In [3]:
# Tracking code
import cv2
from tracker.trackingfilter import FilterSpheroBlueCover, FilterGlow



from tracker.traceable import TraceableObject
from tracker.trackerbase import ColorTracker

from util import Vector2D


In [4]:
# General Python Helper Functions
def readJsonFile(filename):
    '''
        Read a JSON file into a python dict 
    '''
    with open(filename) as data_file:    
        data = json.load(data_file)
    return data

In [5]:
# Config management
PROJ_ROOT = os.getcwd()
ROSTER = os.path.join(PROJ_ROOT, "roster.json")
roster = readJsonFile(ROSTER)

In [6]:
# Class for managing multiple spheros
manager = sphero.SpheroManager()

# Initialize Sphero manager with addresses of local spheros
manager._name_cache = roster

# Number of spheros to connect to
NBOTS = len(roster.keys())

In [7]:
# Add known spheros from roster to the manager
for bt_addr, bt_name in roster.iteritems():
    manager.add_sphero(bt_addr, bt_name)

In [8]:
devices = []
# Verify that these are the robots we are looking for
for name, device in manager._spheros.iteritems():
    print "{}: {}".format(name, device.bt_addr)
    devices.append(device) 

Sphero-RPB: 68:86:E7:05:12:26
Sphero-YPR: 68:86:E7:09:4E:74
Sphero-GRY: 68:86:E7:09:A9:28
Sphero-YYP: 68:86:E7:05:0C:4B
Sphero-RWR: 68:86:E7:09:A2:FE
Sphero-ORG: 68:86:E7:09:A6:FE
Sphero-RYR: 68:86:E7:05:19:AD


In [9]:
bots = [devices[4]]

In [10]:
for bot in bots:
    
    bot.disconnect()
    bot.connect()

Sphero-RWR try 0
An exception of type IOError occurred. Arguments:
('A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond.\r\n',)
Sphero-RWR try 1
Connected!


In [11]:
bots[0].set_rgb(0, 0, 255)




In [12]:
tracker = ColorTracker()
#Track Camera Size: 

tracker.image_size


(640, 480)

In [13]:
# Use blue because it's the most reliable color filter
traceable_blue = TraceableObject("BLUE")
traceable_blue.filter = FilterSpheroBlueCover()

In [14]:
traceable_glow = TraceableObject("GLOW")
traceable_glow.filter = FilterGlow()

In [15]:
# trackable_objects = [traceable_blue]

In [16]:
# traceable_objects = tracker.track_objects(trackable_objects)

In [61]:
# mathematical nuances
def normalize_angle(angle):
    if angle < 0:
        return normalize_angle(-angle)
    
    
    elif angle > 360:
        return angle % 360
    else:
        return angle

def angle_between_points(x1, y1, x2, y2):
    
    return math.degrees(math.atan2( y2 - y1, x2 - x1 ))

In [68]:
# For Debugging
def display_current_view(tracker):
    image = tracker.get_video_frame()
    cv2.imshow("img", image)
    cv2.waitKey(0)
    
# For Navigation
def get_bot_position(bot, traceable_object, tracker, samples=4, debug=False):  
    xSamples = []
    ySamples = []

    traceable_object.screen_size = tracker.image_size
     
    for sample in range(samples):
        print(","),
        
        image = tracker.get_video_frame()
        timestamp = time.time()
        
        if sample > 0: # ignore the first sample
            x, y = tracker._find_traceable_in_image(image, traceable_object) # side effect: adds mask to tracker  
            if x:
                xSamples.append(x)
            if y:
                ySamples.append(y)

            traceable_object.add_tracking(Vector2D(x, y), timestamp)
            cv2.waitKey(1)

    if debug:      
        display_current_view(tracker)
    print "{} | {} ".format(xSamples, ySamples)
        
#     return traceable_object.pos.x, traceable_object.pos.y
    return (sum(xSamples) / len(xSamples)), (sum(ySamples) / len(ySamples))
#     avg_pos = 
#     return traceable_object
    

def calibrate_bot_direction(bot, traceable_object_list, traceable_color, tracker, debug=False):
    """
        Routine for making all robots line up in the same direction
        
    """
#     traceable_object = traceable_object_list[0]
    
    bot.set_rgb(traceable_color[0], traceable_color[1], traceable_color[2])
    bot.set_motion_timeout(1500)
    startX, startY =  get_bot_position(bot, traceable_object_list[0], tracker)

#     traceable_object_list = tracker.track_object(traceable_object_list)
#     startX, startY = traceable_object_list[0].pos.x, traceable_object_list[0].pos.y
    cv2.waitKey(250)
    
    
    if (startX is None) or (startY is None):
        print("Error: Robot not in view")
        return -1
    
    print "Start ({},{})".format( startX, startY)

    bot.roll(60, 0)
    time.sleep(3)
    
    
    
    endX, endY       =  get_bot_position(bot, traceable_object_list[0], tracker)
#     traceable_object_list = tracker.track_object(traceable_object_list)
#     endX, endY = traceable_object_list[0].pos.x, traceable_object_list[0].pos.y
      
    
    print "End   ({},{})".format(   endX, endY)
    offset = normalize_angle(angle_between_points(startX, startY, endX, endY))
    

    print "Angle {}".format(offset) 
#     bot.roll(50, offset)
#     time.sleep(1.5)
#     bot.set_heading(offset)
    
    
    # Turn lights off 
    
#     bot.set_rgb(0, 0, 0)
    
    return offset

In [39]:
# display_current_view(tracker)
tracklist = [traceable_blue]




In [67]:
# Video display

while(True):
#     tracklist = tracker.track_objects(tracklist)
    
    tracker.track_objects(tracklist)
    cv2.waitKey(1)
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):

        break 

        
cv2.destroyAllWindows()


In [ ]:
offsets = []

In [79]:
for bot in bots:
    bot.set_rgb(0, 0, 255)

offset = calibrate_bot_direction(bot, tracklist, [0, 0, 255], tracker, True)


, , , , [251, 251, 251] | [324, 324, 324] 
Start (251,324)
, , , , [335, 335, 335] | [177, 177, 177] 
End   (335,177)
Angle 60.2551187031


In [81]:
bot.roll(50, 0)
time.sleep(1.5)
bot.roll(50, offset)
# bot.set_heading(0)




In [32]:
print traceable_blue.pos




(x: 281, y:156, deg:29.0372889653)


In [70]:
ts.device.roll(40, 180)




In [28]:
print traceable_blue.pos






(x: 444, y:201, deg:24.3563912528)


In [ ]:
for i in range(500):
    print get_bot_position(bot, traceable_blue, tracker, samples=4, debug=False)
    time.sleep(.5)
    

In [118]:
get_bot_position(bot, traceable_blue, tracker, samples=4, debug=False)








[183, 279, 279, 279] | [247, 241, 241, 241] 


(279, 241)

In [63]:
ts = TraceableSphero(bot)



In [53]:
print traceable_blue.pos



(x: 340, y:254, deg:36.7618944194)


In [19]:
ts.tracking_samples = []

image = tracker.get_video_frame()
x, y = tracker._find_traceable_in_image(image, traceable_blue)
ts.add_tracking(Vector2D(x, y), time.time())

print x
print y

143
371


In [20]:
print ts.pos



None


In [56]:
ts.calibrate_direction()

starts calibration
True


In [36]:
get_bot_position(bots[0], traceable_blue, tracker, debug=False)

[454, 454, 454, 454] | [202, 202, 202, 202] 


(454, 202)

[341] | [254] 
[340] | [254] 
Start (341,254)
End   (340,254)
Angle 180.0


180.0

In [85]:
bot.roll(50,270)


In [32]:
bot.roll(50, 135)


time.sleep(1.5)
bots[0].set_heading(0)





In [33]:
bots[0].roll(40, 180)



In [70]:



bots[0].roll(50, 0)


In [109]:
tracker._draw_masks()
cv2.waitKey(1)

Debugging Section

In [43]:
#Stream video live to set 

cam = tracker.cam


while(True):
    # Capture frame-by-frame
    ret, frame = cam.read()

    # Display the resulting frame
    cv2.imshow('frame', frame)
    
    #     cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cv2.destroyAllWindows()

In [ ]:
q